# Title

# Imports

In [11]:
import pandas as pd
import numpy as np

# Read Scenery Dataset

In [2]:
#read the pfac data, which shows the actuall police control area of the county
pfac = pd.read_csv('../data/PFAC.csv')

#read the scenery data 
df = pd.read_csv('scenery.csv', encoding = "ISO-8859-1")
print('Size of the dataset: ' + str(df.shape[0]))

#switch the colomns and rows for pfac data
pfac = pfac.transpose()
pfac.reset_index(inplace = True)

#reset the header of the pfac data
new_header = pfac.iloc[0]
pfac = pfac[1:]
pfac.columns = new_header

print(pfac.shape[1])

Size of the dataset: 212155
43


In [3]:
#find the county
def matchCounty(area):
    for i in pfac.columns:
        if area in list(pfac[i]):
            return i
    return np.nan

# Find Counties represented in dataset

In [4]:
#the unique
len(df['County'].unique())

214

In [5]:
df['County'].unique()

array(['Gloucestershire', 'Cheshire', 'Isle of Man', 'Conwy',
       'North Yorkshire', 'Flintshire/Sir y Fflint', 'Hampshire',
       'Wokingham', 'West Berkshire', 'Kent', 'Poole', 'Dorset',
       'Windsor And Maidenhead', 'Surrey', 'Bracknell Forest',
       'Hertfordshire', 'Bedfordshire', 'Derbyshire', 'Great Britain',
       'Dumfries And Galloway', 'East Riding of Yorkshire',
       'Argyll And Bute', 'North Ayrshire', 'Lincolnshire',
       'Rhondda Cynon Taff/Rhondda Cynon Taf', 'Northamptonshire',
       'Highland', 'Worcestershire', 'Nottinghamshire', 'Northumberland',
       'Na h-Eileanan an Iar', 'Cornwall', 'Pembrokeshire/Sir Benfro',
       'West Sussex', 'Gwynedd', 'Cumbria', 'Devon', 'Essex',
       'The Vale of Glamorgan/Bro Morgannwg', 'Staffordshire',
       'Redcar And Cleveland', 'Scottish Borders', 'Manchester',
       'Suffolk', 'Salford', 'Wirral', 'Bath And North East Somerset',
       'Lancashire', 'County Durham', 'Tameside', 'Warwickshire',
       'Cambri

In [6]:
df['County'] = df['County'].apply(matchCounty)
len(df['County'].unique())

44

In [7]:
sum(df['County'].isnull())

65530

In [8]:
df['County'].unique()

array(['Gloucestershire', 'Cheshire', nan, 'North Wales',
       'North Yorkshire', 'Hampshire', 'Thames Valley', 'Kent', 'Dorset',
       'Surrey', 'Hertfordshire', 'Bedfordshire', 'Derbyshire',
       'Humberside', 'Lincolnshire', 'South Wales', 'Northamptonshire',
       'West Mercia', 'Nottinghamshire', 'Northumbria',
       'Devon and Cornwall', 'Sussex', 'Cumbria', 'Essex',
       'Staffordshire', 'Greater Manchester', 'Suffolk', 'Merseyside',
       'Avon and Somerset', 'Lancashire', 'Durham', 'Warwickshire',
       'Cambridgeshire', 'West Midlands', 'Leicestershire', 'Wiltshire',
       'West Yorkshire', 'Dyfed-Powys', 'Metropolitan Police', 'Norfolk',
       'Cleveland', 'South Yorkshire', 'Gwent', 'City of London'],
      dtype=object)

In [9]:
#df['County'].head(100)

# Read Crime Datasets

In [34]:
number_crime = pd.read_csv('../data/number_crime.csv')
print(len(number_crime))
for c in number_crime.columns:
    print(c)

43
Area Name
Total recorded crime (excluding fraud)
Violence against the person
Homicide
Violence with injury
Violence without injury
Stalking and harassment
Death or serious injury - unlawful driving
Sexual offences
Robbery
Theft offences
Burglary
Residential burglary
Non-residential  burglary
Vehicle offences
Theft from the person
Bicycle theft
Shoplifting
Other theft offences
Criminal damage and arson
Drug offences
Possession of weapons offences
Public order offences
Miscellaneous crimes


In [181]:
number_crime

,Area Name,Total recorded crime (excluding fraud),Violence against the person,Homicide,Violence with injury,Violence without injury,Stalking and harassment,Death or serious injury - unlawful driving,Sexual offences,Robbery,...,Vehicle offences,Theft from the person,Bicycle theft,Shoplifting,Other theft offences,Criminal damage and arson,Drug offences,Possession of weapons offences,Public order offences,Miscellaneous crimes
0,Cleveland,"65,397","20,109",15,"5,858","7,474","6,754",8,"1,755",581,...,"4,333",510,993,"8,867","6,349","9,370","1,298",377,"3,836","1,372"
1,Durham,"62,393","23,908",5,"5,738","8,335","9,817",13,"1,885",242,...,"3,255",285,474,"4,862","5,357","9,585","1,269",290,"5,141","1,735"
2,Northumbria,"160,292","47,904",11,"13,352","19,982","14,522",37,"4,952",855,...,"8,406","1,421","2,027","14,081","16,832","22,742","3,344","1,392","23,401","3,728"
3,Cheshire,"94,201","32,600",19,"10,590","12,892","9,095",4,"3,195",416,...,"3,652",635,"1,577","7,084","7,712","10,162","2,180",532,"17,976","1,842"
4,Cumbria,"35,372","13,244",4,"4,860","5,472","2,903",5,"1,593",123,...,"1,174",197,296,"2,899","3,210","5,458","1,022",252,"3,084",717
5,Greater Manchester,"334,976","97,236",78,"28,581","45,640","22,905",32,"9,656","7,642",...,"31,669","7,351","4,061","17,352","32,096","35,821","4,981","3,542","45,151","8,262"
6,Lancashire,"162,819","62,273",21,"19,920","23,040","19,287",5,"5,131","1,097",...,"10,078","2,000","1,698","9,476","16,378","20,238","2,342",701,"14,938","3,241"
7,Merseyside,"134,164","43,752",21,"14,398","17,163","12,151",19,"3,188","1,462",...,"10,382","2,146","1,890","8,645","10,633","16,328","8,181","1,033","12,689","2,752"
8,Humberside,"98,311","32,747",17,"10,979","12,939","8,791",21,"3,175",973,...,"5,795",702,"2,246","10,356","8,460","12,809","1,374",509,"8,395","2,429"
9,North Yorkshire,"43,762","13,729",5,"5,469","5,478","2,741",36,"1,854",211,...,"2,460",483,"1,177","4,448","4,706","5,993","1,563",309,"2,210",812


In [155]:
rate1000_crime = pd.read_csv('../data/rate1000_crime.csv')
print(rate1000_crime.columns[16])
print(len(rate1000_crime))
for c in rate1000_crime.columns:
    print(c)

Residential burglary6,7,8
43
Area Name
Population figures
 (mid-2017)3  rounded to 100
Household figures
 (mid-2017)4  rounded to 100 
Unnamed: 3
Total recorded crime (excluding fraud5)
Unnamed: 5
Violence against the person
Homicide
Violence with injury
Violence without injury
Stalking
and harassment
Death or serious injury - unlawful driving
Sexual offences
Robbery
Theft offences
Burglary
Residential burglary6,7,8
Residential burglary (households) 6,7,9
Non-residential burglary7,8
Vehicle offences
Theft from the person
Bicycle theft
Shoplifting
All other theft offences
Criminal damage and arson
Drug offences
Possession of weapons offences
Public order offences
Miscellaneous crimes against society


In [182]:
rate1000_crime

,Area Name,Population figures (mid-2017)3 rounded to 100,Household figures (mid-2017)4 rounded to 100,Unnamed: 3,Total recorded crime (excluding fraud5),Unnamed: 5,Violence against the person,Homicide,Violence with injury,Violence without injury,...,Vehicle offences,Theft from the person,Bicycle theft,Shoplifting,All other theft offences,Criminal damage and arson,Drug offences,Possession of weapons offences,Public order offences,Miscellaneous crimes against society
0,Cleveland,"566,200","241,400",NaN,115.5,NaN,35.5,0.0,10.3,13.2,...,7.7,0.9,1.8,15.7,11.2,16.6,2.3,0.7,6.8,2.4
1,Durham,"630,000","278,200",NaN,99.0,NaN,37.9,0.0,9.1,13.2,...,5.2,0.5,0.8,7.7,8.5,15.2,2.0,0.5,8.2,2.8
2,Northumbria,"1,448,600","637,000",NaN,110.7,NaN,33.1,0.0,9.2,13.8,...,5.8,1.0,1.4,9.7,11.6,15.7,2.3,1.0,16.2,2.6
3,Cheshire,"1,054,100","455,500",NaN,89.4,NaN,30.9,0.0,10.0,12.2,...,3.5,0.6,1.5,6.7,7.3,9.6,2.1,0.5,17.1,1.7
4,Cumbria,"498,400","224,200",NaN,71.0,NaN,26.6,0.0,9.8,11.0,...,2.4,0.4,0.6,5.8,6.4,11.0,2.1,0.5,6.2,1.4
5,Greater Manchester,"2,798,800","1,169,100",NaN,119.7,NaN,34.7,0.0,10.2,16.3,...,11.3,2.6,1.5,6.2,11.5,12.8,1.8,1.3,16.1,3.0
6,Lancashire11,"1,490,500","630,600",NaN,109.2,NaN,41.8,0.0,13.4,15.5,...,6.8,1.3,1.1,6.4,11.0,13.6,1.6,0.5,10.0,2.2
7,Merseyside,"1,416,800","621,900",NaN,94.7,NaN,30.9,0.0,10.2,12.1,...,7.3,1.5,1.3,6.1,7.5,11.5,5.8,0.7,9.0,1.9
8,Humberside,"929,900","402,900",NaN,105.7,NaN,35.2,0.0,11.8,13.9,...,6.2,0.8,2.4,11.1,9.1,13.8,1.5,0.5,9.0,2.6
9,North Yorkshire,"819,800","353,200",NaN,53.4,NaN,16.7,0.0,6.7,6.7,...,3.0,0.6,1.4,5.4,5.7,7.3,1.9,0.4,2.7,1.0


In [177]:
knife_crime = pd.read_csv('../data/knife_crime.csv')
knife_crime

,Area Name,TTOSSO:Number of offences involving a knife6,% involving a knife,murder_Number of offences involving a knife6,% involving a knife.1,Threats_Number of offences involving a knife6,% involving a knife.2,Assault_Number of offences involving a knife6,% involving a knife.3,Robbery_Number of offences involving a knife6,% involving a knife.4,Rape_Number of offences involving a knife6,% involving a knife.5,Homicide_Number of offences involving a knife6,% involving a knife.6
0,Cleveland,415,5.0,6.0,-,33.0,18,227,4.0,137,24.0,6.0,0.0,6.0,-
1,Durham,201,3.0,2.0,-,15.0,4,140,3.0,40,17.0,3.0,0.0,1.0,-
2,Northumbria7,887,5.0,7.0,-,61.0,9,597,4.0,205,24.0,13.0,0.0,4.0,-
3,Cheshire,377,3.0,4.0,-,39.0,7,225,2.0,100,24.0,5.0,0.0,4.0,-
4,Cumbria,153,3.0,1.0,-,6.0,9,120,2.0,23,19.0,3.0,0.0,0.0,-
5,Greater Manchester8,"3,614",8.0,30.0,25,263.0,11,"1,515",5.0,"1,728",23.0,69.0,1.0,9.0,13
6,Lancashire,"1,038",4.0,10.0,-,96.0,8,665,3.0,233,22.0,27.0,1.0,7.0,-
7,Merseyside,"1,231",7.0,12.0,-,75.0,11,699,5.0,421,29.0,18.0,1.0,6.0,-
8,Humberside,609,4.0,7.0,-,64.0,12,351,3.0,167,17.0,16.0,1.0,4.0,-
9,North Yorkshire,279,4.0,1.0,-,19.0,14,214,4.0,41,19.0,4.0,0.0,0.0,-


In [184]:
# Merge number_crime with rate1000_crime
# Notes: 
# Population figures and household figures are from mid-2017 and rounded to 100
# Total recorded crime excludes fraud
# Unlawful driving is for death or serious injury
# Residential burglary for households was excluded
datafields = ['Area Name', 'Population figures', 'Household figures',
'Total recorded crime', 'Violence against the person', 'Homicide',
'Violence with injury', 'Violence without injury', 'Stalking and harassment',
'Unlawful driving', 'Sexual offences', 'Robbery',
'Theft offences', 'Burglary', 'Residential burglary',
'Non-residential burglary', 'Vehicle offences', 'Theft from the person',
'Bicycle theft', 'Shoplifting', 'Other theft offences',
'Criminal damage and arson', 'Drug offences', 'Possession of weapons offences',
'Public order offences', 'Miscellaneous crimes']

crime_df = pd.DataFrame(columns = datafields)

def merge_col(name, rate1000_name='', number_name='', both_name=''):
    if both_name != '':
        rate1000_name = both_name
        number_name = both_name
        
    if rate1000_name == '':
        rate1000_name = name
    if number_name == '':
        number_name = name
    crime_df[name] = number_crime[number_name]
    
merge_col('Area Name')
crime_df['Population figures'] = rate1000_crime.iloc[:, 1]
crime_df['Population figures'][44:] = np.NaN
crime_df['Household figures'] = rate1000_crime.iloc[:, 1]
crime_df['Household figures'][44:] = np.NaN
merge_col('Total recorded crime', number_name='Total recorded crime (excluding fraud)', rate1000_name = 'Total recorded crime (excluding fraud5)')
merge_col('Violence against the person')
merge_col('Homicide')
merge_col('Violence with injury')
merge_col('Violence without injury')
merge_col('Stalking and harassment', 'Stalking\nand harassment')
merge_col('Unlawful driving', both_name='Death or serious injury - unlawful driving')
merge_col('Sexual offences')
merge_col('Robbery')
merge_col('Theft offences')
merge_col('Burglary')
merge_col('Residential burglary', rate1000_name = 'Residential burglary6,7,8')
merge_col('Non-residential burglary', number_name='Non-residential  burglary', rate1000_name='Non-residential burglary7,8')
merge_col('Vehicle offences')
merge_col('Theft from the person')
merge_col('Bicycle theft')
merge_col('Shoplifting')
merge_col('Other theft offences', rate1000_name='All other theft offences')
merge_col('Criminal damage and arson')
merge_col('Drug offences')
merge_col('Possession of weapons offences')
merge_col('Public order offences')
merge_col('Miscellaneous crimes', number_name='Miscellaneous crimes', rate1000_name='Miscellaneous crimes against society')
crime_df.reset_index()

,index,Area Name,Population figures,Household figures,Total recorded crime,Violence against the person,Homicide,Violence with injury,Violence without injury,Stalking and harassment,...,Vehicle offences,Theft from the person,Bicycle theft,Shoplifting,Other theft offences,Criminal damage and arson,Drug offences,Possession of weapons offences,Public order offences,Miscellaneous crimes
0,0,Cleveland,"566,200","566,200","65,397","20,109",15,"5,858","7,474","6,754",...,"4,333",510,993,"8,867","6,349","9,370","1,298",377,"3,836","1,372"
1,1,Durham,"630,000","630,000","62,393","23,908",5,"5,738","8,335","9,817",...,"3,255",285,474,"4,862","5,357","9,585","1,269",290,"5,141","1,735"
2,2,Northumbria,"1,448,600","1,448,600","160,292","47,904",11,"13,352","19,982","14,522",...,"8,406","1,421","2,027","14,081","16,832","22,742","3,344","1,392","23,401","3,728"
3,3,Cheshire,"1,054,100","1,054,100","94,201","32,600",19,"10,590","12,892","9,095",...,"3,652",635,"1,577","7,084","7,712","10,162","2,180",532,"17,976","1,842"
4,4,Cumbria,"498,400","498,400","35,372","13,244",4,"4,860","5,472","2,903",...,"1,174",197,296,"2,899","3,210","5,458","1,022",252,"3,084",717
5,5,Greater Manchester,"2,798,800","2,798,800","334,976","97,236",78,"28,581","45,640","22,905",...,"31,669","7,351","4,061","17,352","32,096","35,821","4,981","3,542","45,151","8,262"
6,6,Lancashire,"1,490,500","1,490,500","162,819","62,273",21,"19,920","23,040","19,287",...,"10,078","2,000","1,698","9,476","16,378","20,238","2,342",701,"14,938","3,241"
7,7,Merseyside,"1,416,800","1,416,800","134,164","43,752",21,"14,398","17,163","12,151",...,"10,382","2,146","1,890","8,645","10,633","16,328","8,181","1,033","12,689","2,752"
8,8,Humberside,"929,900","929,900","98,311","32,747",17,"10,979","12,939","8,791",...,"5,795",702,"2,246","10,356","8,460","12,809","1,374",509,"8,395","2,429"
9,9,North Yorkshire,"819,800","819,800","43,762","13,729",5,"5,469","5,478","2,741",...,"2,460",483,"1,177","4,448","4,706","5,993","1,563",309,"2,210",812


In [185]:
print(len(crime_df['Area Name'].unique()))
crime_df[crime_df['Area Name'] == 'Hampshire']

43


,Area Name,Population figures,Household figures,Total recorded crime,Violence against the person,Homicide,Violence with injury,Violence without injury,Stalking and harassment,Unlawful driving,...,Vehicle offences,Theft from the person,Bicycle theft,Shoplifting,Other theft offences,Criminal damage and arson,Drug offences,Possession of weapons offences,Public order offences,Miscellaneous crimes
29,Hampshire,"1,978,800","1,978,800","159,487","55,319",14,"21,387","24,199","9,695",24,...,"12,760","1,272","3,815","12,969","13,559","18,549","3,682","1,621","14,894","2,650"
